In [37]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pickle
import warnings
warnings.filterwarnings('ignore')

In [38]:
# Definimos path
pathdata = '/home/daniu/Documentos/fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/Users/danielarisaro/Documents/Fundar/indice-mercado-trabajo-ingresos/'

In [39]:
with open(pathdata + 'modulos/diccionarios/' + 'diccionario_aglomerados.pickle', 'rb') as file:
    dict_cod_aglomerado = pickle.load(file)

with open(pathdata + 'modulos/diccionarios/' + 'diccionario_provincia.pickle', 'rb') as file:
    dict_cod_provincia = pickle.load(file)

with open(pathdata + 'modulos/diccionarios/' + 'diccionario_aglomerado_provincia.pickle', 'rb') as file:
    map_aglomerado_provincia = pickle.load(file)

with open(pathdata + 'modulos/diccionarios/' + 'diccionario_aglomerado_region.pickle', 'rb') as file:
    map_aglomerado_region = pickle.load(file)

with open(pathdata + 'modulos/diccionarios/' + 'diccionario_provincia_region.pickle', 'rb') as file:
    map_provincia_region = pickle.load(file)

In [40]:
cols = ['PONDERA_new', 'AGLOMERADO', 'CH06']

df_people_pool = pd.read_csv(pathdata + 'data_output/Base_pool_individuos_solo_con_replicas_actuales.csv', low_memory=False, usecols=cols)

In [41]:
# Diccionario regiones ENUT

dict_regiones_ENUT = {1: 'Gran Buenos Aires',
                    2: 'Pampeana',
                    3: 'Noroeste',
                    4: 'Noreste',
                    5: 'Cuyo',
                    6: 'Patagonia'}

In [42]:
# Cargo ENUT
pathfolder_ENUT =  pathdata + 'data_input/ENUT_2021/Bases de datos/'
filename = 'enut2021_base.txt'
df_enut = pd.read_csv(pathfolder_ENUT + filename, sep='|')
columnas = pd.DataFrame(df_enut_base.columns)

## Hago todo el calculo by myself

In [5]:
## Calculo coeficientes para imputar a PBA
df_temp = df_people_pool[df_people_pool['CH06']>14]
df_pob = df_temp.groupby(['AGLOMERADO']).sum(['PONDERA_new'])
df_pob.index = df_pob.index.map(dict_cod_aglomerado)
df_pob['PROVINCIA'] = df_pob.index.map(map_aglomerado_provincia)
pob_pba = df_pob[df_pob['PROVINCIA']=='Buenos Aires']['PONDERA_new'].sum()
gba = df_pob[df_pob['PROVINCIA']=='Buenos Aires'].loc['Partidos del GBA', 'PONDERA_new']
resto_pba = df_pob[df_pob['PROVINCIA']=='Buenos Aires'].loc[['Gran La Plata', 'Bahía Blanca - Cerri', 'Mar del Plata', 'Resto Buenos Aires'], 'PONDERA_new'].sum()
coef_gba = gba/pob_pba
coef_resto_pba = resto_pba/pob_pba


In [8]:
# Calculo TNR
pondera = 'WPER'
df_temp = df_enut_base[df_enut_base['EDAD_SEL']>=14]
df_selected = df_temp[(~df_temp['TSS_GRANGRUPO_TNR'].isna())][['TSS_GRANGRUPO_TNR', 'SEXO_SEL', 'REGION', pondera]]
df_selected['Multiplication'] = df_selected['TSS_GRANGRUPO_TNR'] * df_selected[pondera]

numerador = df_selected.groupby(['SEXO_SEL', 'REGION'])['Multiplication'].sum().unstack(level=0)
denominador = df_selected.groupby(['SEXO_SEL', 'REGION'])[pondera].sum().unstack(level=0)
tasa = numerador.div(denominador, fill_value=np.nan)    
tasa = tasa.rename(columns= {2:'Varon', 1:'Mujer'})

size = df_temp[(~df_temp['TSS_GRANGRUPO_TNR'].isna())].groupby(['REGION', 'SEXO_SEL']).size().unstack(level=1)
size = size.rename(columns= {2:'N_v', 1:'N_m'})

n_pob = df_temp[(~df_temp['TSS_GRANGRUPO_TNR'].isna())].groupby(['REGION', 'SEXO_SEL'])[pondera].sum().unstack(level=1)
n_pob = n_pob.rename(columns= {2:'N_pob_v', 1:'N_pob_m'})

size[['N_pob_v', 'N_pob_m']] = n_pob[['N_pob_v', 'N_pob_m']]
size['N_pob_tot'] = n_pob['N_pob_v'] + n_pob['N_pob_m'] 

ratio = tasa['Varon'] / tasa['Mujer']

error = pd.DataFrame(index=size.index, columns=['LI', 'LS', 'ME', 'ER'])

ratio_minutos_promedio_no_pago = pd.concat([tasa, size, ratio*100, error], axis=1)
ratio_minutos_promedio_no_pago.rename(columns={0: 'Tiempo TNR'}, inplace=True)

ratio_minutos_promedio_no_pago.index = ratio_minutos_promedio_no_pago.index.map(dict_regiones_ENUT)

In [9]:
# Imputo de regiones a provincias
map_provincia_region['Ciudad Autónoma de Buenos Aires'] = 'Gran Buenos Aires'
provincias = list(map_provincia_region.keys())
regiones = [map_provincia_region[provincia] for provincia in provincias]

ratio_minutos_promedio_no_pago_provincial = pd.DataFrame(index=provincias, columns=['Mujer', 'Varon'])
for provincia in provincias:
    region = map_provincia_region[provincia]
    ratio_minutos_promedio_no_pago_provincial.loc[provincia,:] = ratio_minutos_promedio_no_pago.loc[region,['Mujer', 'Varon']]

ratio_minutos_promedio_no_pago_provincial.index.name = 'Provincia'


In [10]:
## Corrijo el caso de PBA
ratio_minutos_promedio_no_pago_provincial.loc['Buenos Aires', ['Mujer', 'Varon']] = ratio_minutos_promedio_no_pago.loc['Gran Buenos Aires', ['Mujer', 'Varon']]*coef_gba + \
                                        ratio_minutos_promedio_no_pago.loc['Pampeana', ['Mujer', 'Varon']]*coef_resto_pba

# Calculo ratio
ratio_minutos_promedio_no_pago_provincial['Tiempo TNR'] = ratio_minutos_promedio_no_pago_provincial['Varon'] / ratio_minutos_promedio_no_pago_provincial['Mujer']*100
ratio_minutos_promedio_no_pago_provincial = ratio_minutos_promedio_no_pago_provincial.rename(index={'Ciudad Autónoma de Buenos Aires':'CABA', 'Tierra del Fuego, Antártida e Islas del Atlántico Sur':'TdF'})

# Exporto csv
ratio_minutos_promedio_no_pago = ratio_minutos_promedio_no_pago.sort_index()
ratio_minutos_promedio_no_pago_provincial = ratio_minutos_promedio_no_pago_provincial.sort_index()


In [ ]:
ratio_minutos_promedio_no_pago_provincial.to_csv(pathdata + 'data_output/indicadores/05_tiempo_no_remunerado.csv')
ratio_minutos_promedio_no_pago.to_csv(pathdata + 'data_output/indicadores/05_tiempo_no_remunerado_regional.csv')